Author: Dennis Trollsfjord

Followed tutorial by tensorflow found [here](https://www.tensorflow.org/tutorials/images/classification), with modifications adapted for our implementation. 

In [30]:
%%capture
pip install matplotlib numpy tensorflow tensorflow_datasets --user

In [31]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential

import tensorflow_datasets as tfds  


[Tensorflow datasets](https://www.tensorflow.org/datasets/overview)

[Dataset](https://www.tensorflow.org/datasets/catalog/rock_paper_scissors)

In [46]:
# Load dataset whole dataset.
dataset, dataset_info = tfds.load(
    name='rock_paper_scissors', 
    data_dir='dataset', 
    with_info=True,
    as_supervised=True) 
assert isinstance(dataset, dict) 
assert isinstance(dataset_info, tfds.core.dataset_info.DatasetInfo)
dataset_label_size = dataset_info.features['label'].num_classes
print(f"dataset name: {dataset_info.name}")  
print(f"dataset size: {dataset_info.dataset_size}")
print(f"number of labels: {dataset_label_size}")
print(f"dataset labels: {dataset_info.features['label'].names}")

# Train dataset.
dataset_train = dataset['train']
dataset_train_size = dataset_info.splits['train'].num_examples
print(f"train dataset size: {dataset_train_size}") 

# Test dataset.
dataset_test = dataset['test']
dataset_test_size = dataset_info.splits['test'].num_examples
print(f"test dataset size: {dataset_test_size}")  

dataset name: rock_paper_scissors
dataset size: 219.23 MiB
number of labels: 3
dataset labels: ['rock', 'paper', 'scissors']
train dataset size: 2520
test dataset size: 372
